# Подробное руководство по ноутбуку

Этот ноутбук формирует суточные погодные признаки для каждого МКД из синтетического набора данных.
Следуйте шагам ниже, чтобы воспроизводимо получить таблицу `weather_features` и использовать её в `train.ipynb`.

1. **Импорты и базовые настройки.** Загружаем необходимые библиотеки и задаём директорию с данными.
2. **Загрузка телеметрии.** Читаем справочник МКД и формируем координатные ключи на основе широты/долготы.
3. **Календарная сетка.** Строим полный диапазон дат для отопительного года и подготавливаем признаки сезонности.
4. **Генерация погодных показателей.** Создаём синтетические температуры, осадки, влажность и ветровые нагрузки для каждого дома.
5. **Эксплуатационные периоды.** Назначаем режим (отопительный, переходный, летний) и считаем сводную статистику.
6. **Проверки качества.** Убеждаемся, что все ключевые столбцы присутствуют и пригодны для объединения с телеметрией.
7. **Экспорт.** Сохраняем параметризованный CSV/Parquet с ключами `date` + `geo_key` для повторного использования.

### Сценарий подготовки погодных признаков

Вместо координат Open-Meteo мы используем реальные адреса и широты/долготы из синтетического телеметрического набора.
Для каждого дома генерируются суточные значения температуры, осадков и ветра, а также индикатор теплопотерь при отрицательных температурах. Эти признаки соответствуют потребностям ГВС (отопление, циркуляция и теплопотери).

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

In [ ]:
# Загружаем телеметрию и готовим справочник по домам
DATA_DIR = Path('data')
telemetry_path = DATA_DIR / 'telemetry.csv'
telemetry = pd.read_csv(telemetry_path, parse_dates=['date'])
telemetry['geo_key'] = telemetry['mkd_lat'].round(4).astype(str) + '_' + telemetry['mkd_lon'].round(4).astype(str)
buildings = telemetry[
    ['mkd_id', 'mkd_address', 'district', 'mkd_lat', 'mkd_lon', 'geo_key']
].drop_duplicates().reset_index(drop=True)
telemetry.head()

In [ ]:
# Формируем календарь суточных наблюдений
start_date = telemetry['date'].min().replace(day=1)
end_date = telemetry['date'].max()
calendar = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='D')})
calendar['dayofyear'] = calendar['date'].dt.dayofyear
calendar['month'] = calendar['date'].dt.month
calendar.head()

In [ ]:
# Генерируем погодные признаки на каждый дом и день
rng = np.random.default_rng(2024)
weather_daily = (
    buildings.assign(key=1)
    .merge(calendar.assign(key=1), on='key')
    .drop(columns='key')
    .sort_values(['mkd_id', 'date'])
    .reset_index(drop=True)
)

seasonal_component = -9 * np.cos(2 * np.pi * (weather_daily['dayofyear'] - 30) / 365) - 3
building_offset = weather_daily['mkd_id'].map({
    row.mkd_id: rng.normal(0, 0.8)
    for row in buildings.itertuples(index=False)
})
noise = rng.normal(0, 1.5, size=len(weather_daily))
weather_daily['avg_temp_c'] = seasonal_component + building_offset + noise

low_spread = rng.uniform(2.5, 6.5, size=len(weather_daily))
high_spread = rng.uniform(2.0, 5.0, size=len(weather_daily))
weather_daily['min_temp_c'] = weather_daily['avg_temp_c'] - low_spread
weather_daily['max_temp_c'] = weather_daily['avg_temp_c'] + high_spread

BASE_TEMP = 18.0
weather_daily['heating_degree_days'] = (BASE_TEMP - weather_daily['avg_temp_c']).clip(lower=0)

precip_shape = np.where(weather_daily['month'].isin([6, 7, 8]), 1.8, 1.2)
precip_scale = np.where(weather_daily['month'].isin([6, 7, 8]), 1.1, 0.8)
weather_daily['precipitation_mm'] = rng.gamma(shape=precip_shape, scale=precip_scale)

humidity_base = np.where(weather_daily['avg_temp_c'] < 0, 80, 65)
weather_daily['relative_humidity'] = np.clip(
    humidity_base + rng.normal(0, 7, size=len(weather_daily)), 40, 98
)

cloud_base = np.where(weather_daily['month'].isin([6, 7, 8]), 7, 11)
weather_daily['cloudiness_hours'] = np.clip(
    cloud_base + rng.normal(0, 3, size=len(weather_daily)), 0, 24
)

wind_base = np.where(weather_daily['avg_temp_c'] < 0, 4.5, 3.2)
weather_daily['wind_speed_ms'] = np.clip(
    wind_base + rng.normal(0, 1.2, size=len(weather_daily)), 0.5, 10
)
weather_daily['wind_load_subzero'] = weather_daily['wind_speed_ms'] * (-weather_daily['avg_temp_c']).clip(lower=0)

weather_daily.head()

In [ ]:
# Определяем эксплуатационные периоды и форматируем значения
weather_daily['operational_period'] = np.select(
    [weather_daily['month'].isin([10, 11, 12, 1, 2, 3, 4]), weather_daily['month'].isin([5, 9])],
    ['Отопительный сезон', 'Переходный период'],
    default='Летний режим'
)
round_map = {
    'avg_temp_c': 2, 'min_temp_c': 2, 'max_temp_c': 2,
    'heating_degree_days': 2, 'precipitation_mm': 2,
    'relative_humidity': 1, 'cloudiness_hours': 1,
    'wind_speed_ms': 2, 'wind_load_subzero': 3
}
for col, digits in round_map.items():
    weather_daily[col] = weather_daily[col].round(digits)
weather_daily.head()

### Сводная статистика по эксплуатационным периодам

Эта таблица помогает убедиться, что синтетика отражает реальные тренды: зимой значения HDD и ветровой нагрузки выше, летом — больше осадков и продолжительность облачности.

In [ ]:
period_stats = (
    weather_daily
    .groupby(['mkd_id', 'operational_period'])
    .agg(
        days=('date', 'nunique'),
        avg_temp=('avg_temp_c', 'mean'),
        mean_hdd=('heating_degree_days', 'mean'),
        mean_precip=('precipitation_mm', 'mean'),
        wind_load=('wind_load_subzero', 'mean')
    )
    .reset_index()
)
period_stats

### Критерии отбора признаков для прогноза ГВС

- **Heating Degree Days** — напрямую отражает нагрузку на циркуляцию ГВС и потребность в догреве воды.
- **Wind Load Subzero** — индикатор теплопотерь стояков при отрицательных температурах и высоком ветре.
- **Cloudiness Hours и относительная влажность** — косвенно описывают теплопритоки и испарение, влияющие на баланс ГВС.
- **Эксплуатационный период** — разделяет режимы работы сети (отопительный / переходный / летний) для корректного сравнения потребления.
- **Осадки (precipitation_mm)** — учитывают дополнительное охлаждение и влияние на теплоизоляцию вводов.

In [ ]:
# Финальная таблица погодных признаков
weather_features = weather_daily[
    ['date', 'mkd_id', 'district', 'mkd_lat', 'mkd_lon', 'geo_key',
     'avg_temp_c', 'min_temp_c', 'max_temp_c', 'heating_degree_days',
     'precipitation_mm', 'relative_humidity', 'cloudiness_hours',
     'wind_speed_ms', 'wind_load_subzero', 'operational_period']
].copy()
required_columns = {
    'date', 'geo_key', 'avg_temp_c', 'min_temp_c', 'max_temp_c',
    'heating_degree_days', 'precipitation_mm', 'relative_humidity',
    'cloudiness_hours', 'wind_speed_ms', 'wind_load_subzero',
    'operational_period'
}
missing = required_columns - set(weather_features.columns)
if missing:
    raise ValueError(f'Отсутствуют обязательные столбцы: {missing}')
weather_features.head()

In [ ]:
# Сохраняем подготовленные данные
OUTPUT_CSV = DATA_DIR / 'weather_features.csv'
OUTPUT_PARQUET = OUTPUT_CSV.with_suffix('.parquet')
weather_features.to_csv(OUTPUT_CSV, index=False)
weather_features.to_parquet(OUTPUT_PARQUET, index=False)
OUTPUT_CSV, OUTPUT_PARQUET

In [ ]:
# Контрольная загрузка файлов для ноутбука train.ipynb
check_csv = pd.read_csv(OUTPUT_CSV, parse_dates=['date'])
check_parquet = pd.read_parquet(OUTPUT_PARQUET)
check_csv.head(), check_parquet.head()